# Idea

### Режимы работы библиотеки:

1. На вход ей подается текст и на выход получаем фонемы (разные прочтения)
2. На вход подаем фонемы двух типов и получаем их разницу
3. Мы подаем текст/фонемы и логиты (вероятности каждой фонемы в каждом звуковом окне) и получаем транскрипцию

### Хотим учесть:

- cmudict + lextool для первичного преобразования текста в фонемы по словарю/эвристикам
- функции для работы с фонемами - трансляция 61-48-39, one-letter-encoding, diff функции и подсчет операций
- витерби для третьего режима
- эвристики на стыке слов и похожие звуки
- сравнение с фонемами тимита
- разные варианты произношения слова

На будущее:
- сравнить с https://github.com/bootphon/phonemizer на различных бэкендах - используют ли они тоже cmudict внутри или какой-то свой фонемайзер
- ударения - как отдельных слов так и целиком в предложении

### Начальные статы сравнения транскрипций словарных с тимитом:

```
sum(stats["cer"])/len(stats["cer"])
0.24904557704602598

sorted(stats["cer"])[int(len(stats["cer"])/2)]
0.25
```

In [ ]:
# 1. Перевести разметку тимита при помощи cmudict

import torch
from glob import glob
from soundfile import read as read_sound

import editdistance



cmu = {}
for word in open("/home/gazay/code/jongleur/cmudict.dict.txt").read().split("\n"):
    word_parts = word.split(' ')
    cmu[word_parts[0]] = [''.join(filter(lambda x: not x.isdigit(), phn.lower())) for phn in word_parts[1:]]
    
class TimitDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        self.path = path
        self.wavs = glob(path + '/*/*/*/*.wav')
        
    def __len__(self):
        return len(self.wavs)
    
    def __read_phns__(self, path):
        raw_phns = open(path).read()
        phns = [phn.split(' ')[-1] for phn in raw_phns.split('\n') if len(phn)]
        phns = remap(phns)
        phns = [phn for phn in phns if phn != 'sil']
        return phns
    
    def __read_txt__(self, path):
        raw_text = open(path).read()
        return raw_text.split(' ', 2)[-1]
    
    def __getitem__(self, idx):
        wav_path = self.wavs[idx]
        phn_path = wav_path.replace('.WAV.wav', '.PHN')
        txt_path = wav_path.replace('.WAV.wav', '.TXT')
        wav = read_sound(wav_path)
        phns = self.__read_phns__(phn_path)
        text = self.__read_txt__(txt_path)
        _text = text.lower()   \
            .replace('.', '')  \
            .replace("\n", '') \
            .replace('?', '')  \
            .replace(',', '')  \
            .replace(';', '')  \
            .replace(':', '')  \
            .replace('"', '')  \
            .replace('!', '')  \
            .replace('-', ' ') \
            .split(' ')
        cmu_phns = []
        
        cer_to_null = False
        for word in _text:
            if word not in cmu:
                print("word not in dict: ", word)
                cer_to_null = True
                continue
            cmu_phns.extend(cmu[word])
        cer = editdistance.eval(phns, cmu_phns)/len(phns)
        if cer_to_null:
            cer = 0
        
        _phns = single_char_encode(phns)
        _cmu_phns = single_char_encode(cmu_phns)
        return {"wav": wav, "orig_phns": phns, "orig_cmu_phns": cmu_phns, "phns": _phns, "text": text, "cmu_phns": _cmu_phns, "cer": cer}

In [ ]:
# for unknown words http://www.speech.cs.cmu.edu/tools/lextool.html

import os
import IPython
from diff_match_patch import diff_match_patch
DIFFER = diff_match_patch()

def show_diff(r, t):
    html_diffs = DIFFER.diff_main(r, t)
    display(IPython.display.HTML(DIFFER.diff_prettyHtml(html_diffs)))
    
def diff(item):
    print(item["text"])
    show_diff(item["phns"], item["cmu_phns"])
    
def load_phone_map():
    with open('/home/gazay/code/jongleur/phones.61-48-39.map', 'r') as fid:
        lines = (l.strip().split() for l in fid)
        lines = [l for l in lines if len(l) == 3]
    m61_48 = {l[0] : l[1] for l in lines}
    m48_39 = {l[1] : l[2] for l in lines}
    return m61_48, m48_39

m61_48, m48_39 = load_phone_map()

def remap_48_to_39(data):
    return [m48_39[p] for p in data if p in m48_39]

def remap_61_to_48(data):
    return [m61_48[p] for p in data if p in m61_48]

# TODO: document phonems in different models/datasets
def remap(data):
    result = []
    for phn in data:
        # dx is missing from awni 39 phonemes
        if phn == 'dx':
            result.append('d')
        elif phn == 'sil': # in case we override phoneme target and use SIL symbol
            result.append(phn)
        elif phn != 'q':
            result.append(m48_39[m61_48[phn]])
    return result

def single_char_encode(phns):
    return ''.join([one_letter_encoding[phn] for phn in phns])

one_letter_encoding = {
    'aa': 'a',
    'ae': '@',
    'ah': 'A',
    'ao': 'c',
    'aw': 'W',
    'ax': 'x',
    'ay': 'Y',
    'b': 'b',
    'ch': 'C',
    'cl': '-',
    'd': 'd',
    'dh': 'D',
    'dx': 'F',
    'eh': 'E',
    'el': 'L',
    'en': 'N',
    'epi': '=',
    'er': 'R',
    'ey': 'e',
    'f': 'f',
    'g': 'g',
    'hh': 'h',
    'ih': 'I',
    'ix': 'X',
    'iy': 'i',
    'jh': 'J',
    'k': 'k',
    'l': 'l',
    'm': 'm',
    'n': 'n',
    'ng': 'G',
    'ow': 'o',
    'oy': 'O',
    'p': 'p',
    'r': 'r',
    's': 's',
    'sh': 'S',
    'sil': '_',
    't': 't',
    'th': 'T',
    'uh': 'U',
    'uw': 'u',
    'v': 'v',
    'vcl': '+',
    'w': 'w',
    'y': 'y',
    'z': 'z',
    'zh': 'Z'
}

In [ ]:
ds = TimitDataset(path="/home/gazay/datasets/TIMIT")

In [ ]:
item = next(iter(ds))
item["text"]

In [ ]:
# item["orig_phns"]

In [ ]:
stats = {
    "len": [],
    "cer": []
}

for item in ds:
    stats["len"].append(len(item["phns"]))
    stats["cer"].append(item["cer"])

In [ ]:
cmu["and"]

In [ ]:
show_diff(item["phns"], item["cmu_phns"])

In [ ]:
sum(stats["cer"])/len(stats["cer"])

In [ ]:
sorted(stats["cer"])[int(len(stats["cer"])/2)]

In [ ]:
stats["cer"][:10]

In [ ]:
cmu['spend']

In [ ]:
item = ds.__getitem__(7)
diff(item)

In [ ]:
item['phns']

In [ ]:
item['orig_phns']